Red neuronal convolucional para problema de clasificación multiclase con keras

Utilizamos un dataset de la nube, de google colab

In [ ]:
# Utilizamos un dataset de la nube, https://www.kaggle.com/datasets/puneet6060/intel-image-classification
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

In [ ]:
# Important imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
import random
from os import listdir
from sklearn.preprocessing import  LabelBinarizer
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, array_to_img
from keras.optimizers import Adam
from PIL import Image
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense, LeakyReLU
from sklearn.model_selection import train_test_split

In [ ]:
# mostramos las primeras 25 imagenes
plt.figure(figsize=(11,11))
path = "/content/drive/My Drive/Intel Image Dataset/Intel Image Dataset/mountain"
for i in range(1,26):
    plt.subplot(5,5,i)
    plt.tight_layout()
    rand_img = imread(path +'/'+ random.choice(sorted(listdir(path))))
    plt.imshow(rand_img)
    plt.title('mountain')
    plt.xlabel(rand_img.shape[1], fontsize = 10)
    plt.ylabel(rand_img.shape[0], fontsize = 10)

In [ ]:
# creación de listas en el directorio raiz
dir = "/content/drive/My Drive/Intel Image Dataset/Intel Image Dataset" 
root_dir = listdir(dir)
image_list, label_list = [], []

In [ ]:
# Convertimos imagenes a numeros (numpy) en listas de imagenes y etiquetas
for directory in root_dir:
  for files in listdir(f"{dir}/{directory}"):
    image_path = f"{dir}/{directory}/{files}"
    image = Image.open(image_path)
    image = image.resize((150,150)) # Estandarizamos las dimenciones de las imagenes
    image = img_to_array(image)
    image_list.append(image)
    label_list.append(directory)

In [ ]:
# Numero de clases posibles e imagen por clase
label_counts = pd.DataFrame(label_list).value_counts()

num_classes = len(label_counts)
print("clases",label_counts)
print("numero por clases",num_classes)

Número de clases que se utilizarán más adelante en la arquitectura del modelo  

Tamaño para capa de entrada

In [ ]:
np.array(image_list).shape
label_list = np.array(label_list)
label_list.shape

In [ ]:
# separameos test y train
x_train, x_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.2, random_state = 10) 

In [ ]:
# Normalizamos
# Binarizing labels cambia 255 canales de color a 1 y 0 escala de grises
x_train = np.array(x_train, dtype=np.float16) / 225.0
x_test = np.array(x_test, dtype=np.float16) / 225.0
x_train = x_train.reshape( -1, 150,150,3)
x_test = x_test.reshape( -1, 150,150,3)

One hot enconder para las etiquetas

In [ ]:
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)
print(lb.classes_)

In [ ]:
# Dividimos el entrenamiento para validación tambien
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

In [ ]:
# Arquitectura del modelo
model = Sequential([
        Conv2D(16, kernel_size = (3,3), input_shape = (150,150,3)),
        BatchNormalization(),
        LeakyReLU(),
          
        Conv2D(32, kernel_size = (3,3)),
        BatchNormalization(),
        LeakyReLU(),
        MaxPooling2D(5,5),
        
        Conv2D(64, kernel_size = (3,3)),
        BatchNormalization(),
        LeakyReLU(),
        
        Conv2D(128, kernel_size = (3,3)),
        BatchNormalization(),
        LeakyReLU(),
        MaxPooling2D(5,5),

        Flatten(),
    
        Dense(64),
        Dropout(rate = 0.2),
        BatchNormalization(),
        LeakyReLU(),
        
        Dense(32),
        Dropout(rate = 0.2),
        BatchNormalization(),
        LeakyReLU(),
    
        Dense(16),
        Dropout(rate = 0.2),
        BatchNormalization(),
        LeakyReLU(1),
    
        Dense(6, activation = 'softmax')    
        ])
model.summary()

Hemos utilizado diferentes tipos de capas según sus características a saber:
BatchNormalization 
(La normalización por lotes es una técnica para el entrenamiento de redes neuronales muy profundas que normaliza las entradas a una capa para cada minilote), 

LeakyRelu 
(El Leaky ReLU modifica la función para permitir pequeños valores negativos cuando la entrada es menor que cero), 

Conv_2d 
(Se utiliza para crear un kernel convolucional que se convoluciona con la capa de entrada para producir el tensor de salida),

max_pooling2d 
(Es una técnica de downsampling que saca el valor máximo sobre la ventana definida por poolsize), 

flatten 
(Aplana la entrada y crea una salida 1D), 

Dense 
(La capa Dense produce la salida como el producto punto de la entrada y el kernel). 

En la última capa utilizaremos softmax como función de activación porque se trata de un problema de clasificación de varias clases.

In [ ]:
# Compilamos el modelo
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0005),metrics=['accuracy'])
"""
3 parámetros: 
pérdida, optimizador y métrica. 
Aquí usaremos la pérdida como categorical_crossentropy, el optimizador como Adam y la métrica como precisión.
"""

In [ ]:
# Entrenamiento
epochs = 70
batch_size = 128
history = model.fit(
    x_train, 
    y_train, 
    batch_size = batch_size, 
    epochs = epochs, 
    validation_data = (x_val, y_val)
    )

Se puede intentar utilizar un mayor número de épocas para aumentar la precisión. Durante cada época podemos ver cómo se comporta el modelo viendo la precisión del entrenamiento y de la validación.

In [ ]:
# Guardamos el modelo
model.save("/content/drive/My Drive/intel_image.h5")

In [ ]:
#Entrenamiento en el tiempo
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], color='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
#Funcion de perdida
plt.figure(figsize=(12, 5))
plt.plot(history.history['loss'], color='r')
plt.plot(history.history['val_loss'], color='b')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
# Evaluacion del modelo
y_pred = model.predict(x_test)

In [ ]:
img = array_to_img(x_test[1])
img

visualiamos imagenes original vs predicha

In [ ]:
labels = lb.classes_
print(labels)
print("Originally : ",labels[np.argmax(y_test[1])])
print("Predicted : ",labels[np.argmax(y_pred[1])])